In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df = pd.read_csv("golden_mine.csv")

In [ ]:
df.head()

,Payments,Costs,Profit,Age,Gender,AgeGroup
0,9400,8100,1300,19,male,18-24
1,26600,13900,12700,29,male,25-39
2,9750,6750,3000,20,male,18-24
3,29700,11200,18500,18,female,18-24
4,49700,10450,39250,21,female,18-24


In [ ]:
df.shape

(8715, 6)

Количество клиентов - 8715

In [ ]:
df_profit = df['Profit'].astype(np.int64)
print(df_profit.describe())

count    8.715000e+03
mean     1.540725e+04
std      1.271005e+05
min     -9.643950e+06
25%      2.500000e+03
50%      7.850000e+03
75%      2.375000e+04
max      6.643950e+06
Name: Profit, dtype: float64


Заметим, что данные очень неоднородны, потому что стандартное отклонение = 127100, при условии, что среднее значение 15407. Среднее значение не показывают реальную ситуацию, потому что есть клиенты с крайне большой прибылью, что мешает корректности статистических измерений. Именно из-за этих выбросов медиана и среднее значение очень разнятся.

Еще заметим, что есть отрицательная прибыль(убыточные клиенты)

In [ ]:
negative_profit = df[df['Profit'] < 0]
print("АНАЛИЗ УБЫТОЧНЫХ КЛИЕНТОВ:")
print(f"Количество: {len(negative_profit):,}")
print(f"Доля от общего числа: {len(negative_profit)/len(df)*100:.1f}%")
print(f"Средний убыток: {negative_profit['Profit'].mean():,.0f} руб.")
print(f"Максимальный убыток: {negative_profit['Profit'].min():,.0f} руб.")

АНАЛИЗ УБЫТОЧНЫХ КЛИЕНТОВ:
Количество: 712
Доля от общего числа: 8.2%
Средний убыток: -14,895 руб.
Максимальный убыток: -9,643,950 руб.


In [ ]:
negative_profit.head()

,Payments,Costs,Profit,Age,Gender,AgeGroup
35,11300,11700,-400,38,Female,25-39
43,8250,8450,-200,19,male,18-24
48,8500,9100,-600,20,male,18-24
56,6600,9300,-2700,18,male,18-24
58,6850,8650,-1800,20,male,18-24


In [ ]:
print(df[df['Profit'] == -9643950])

      Payments    Costs   Profit  Age Gender AgeGroup
2759     17750  9661700 -9643950   35   male    25-39


In [ ]:
print(df[df['Profit'] <= negative_profit['Profit'].mean()])

      Payments    Costs   Profit  Age Gender AgeGroup
2759     17750  9661700 -9643950   35   male    25-39


Удалим эту аномальную строчку.

In [ ]:
extreme_index = df[df['Profit'] == -9643950].index
df = df.drop(extreme_index).copy()

In [ ]:
df['Profit'].min()

-10200

In [ ]:
df['Age'].describe()

,Age
count,8714.000000
mean,22.391554
std,5.876615
min,18.000000
25%,18.000000
50%,19.000000
75%,25.000000
max,49.000000


In [ ]:
df['Payments'].describe()

,Payments
count,8.714000e+03
mean,3.009929e+04
std,7.605549e+04
min,3.750000e+03
25%,1.080000e+04
50%,1.985000e+04
75%,4.005000e+04
max,6.654600e+06


In [ ]:
df['Gender'].unique()

array(['male', 'female', 'Female', 'FEMALE'], dtype=object)

In [ ]:
df['Gender'] = df['Gender'].str.lower().str.strip()

In [ ]:
df['Gender'].unique()

array(['male', 'female'], dtype=object)

In [ ]:
null_columns = df.columns[df.isnull().any()].tolist()
print(f"Столбцы с пропусками: {null_columns}")

Столбцы с пропусками: []


In [ ]:
calculation_diff = df['Payments'] - df['Costs'] - df['Profit']
calculation_errors = df[abs(calculation_diff) > 1]
print(f"Записей с несовпадением Profit = Payments - Costs: {len(calculation_errors)}")

Записей с несовпадением Profit = Payments - Costs: 0


In [ ]:
df.describe()

,Payments,Costs,Profit,Age
count,8.714000e+03,8714.000000,8.714000e+03,8714.000000
mean,3.009929e+04,13583.555199,1.651573e+04,22.391554
std,7.605549e+04,8038.908871,7.380149e+04,5.876615
min,3.750000e+03,3500.000000,-1.020000e+04,18.000000
25%,1.080000e+04,8350.000000,2.500000e+03,18.000000
50%,1.985000e+04,11450.000000,7.850000e+03,19.000000
75%,4.005000e+04,15350.000000,2.375000e+04,25.000000
max,6.654600e+06,54350.000000,6.643950e+06,49.000000


In [ ]:
df['Profit'].sum()

np.int64(143918100)

In [ ]:
mean_profit = df['Profit'].mean()
median_profit = df['Profit'].median()
print(mean_profit, median_profit)

16515.733302731238 7850.0


In [ ]:
ages_df = df.groupby("AgeGroup")['Profit'].agg([
    ('Количество', 'count'),
    ('Средняя прибыль', 'mean'),
    ('Медианная прибыль', 'median'),
    ('Общая прибыль', 'sum')
]).round(0)
print(ages_df)

          Количество  Средняя прибыль  Медианная прибыль  Общая прибыль
AgeGroup                                                               
18-24           6391          18724.0             8350.0      119667000
25-39           2232          10875.0             8025.0       24272350
40+               91           -234.0             -350.0         -21250


In [ ]:
youth_df = df[df['AgeGroup'] == '18-24']

youth_comparison = youth_df.groupby('Gender')['Profit'].agg([
    ('Количество', 'count'),
    ('Средняя прибыль', 'mean'),
    ('Медианная прибыль', 'median'),
    ('Общая прибыль', 'sum')
]).round(0)


print(youth_comparison)

        Количество  Средняя прибыль  Медианная прибыль  Общая прибыль
Gender                                                               
female        2712          26278.0            26100.0       71266950
male          3679          13156.0             2500.0       48400050


In [ ]:
category_analysis = df.groupby(['AgeGroup', 'Gender']).agg({
    'Profit': [
        ('Клиентов', 'count'),
        ('Средняя_прибыль', 'mean'),
        ('Медианная_прибыль', 'median'),
        ('Общая_прибыль', 'sum')
    ]
}).round(0)

print(category_analysis)

                  Profit                                                
                Клиентов Средняя_прибыль Медианная_прибыль Общая_прибыль
AgeGroup Gender                                                         
18-24    female     2712         26278.0           26100.0      71266950
         male       3679         13156.0            2500.0      48400050
25-39    female     1212          7879.0            7900.0       9549950
         male       1020         14434.0            8200.0      14722400
40+      female       72          -344.0            -500.0        -24750
         male         19           184.0               0.0          3500


In [ ]:
def create_detailed_age_groups(age):
    if age <= 20: return '18-20'
    elif age <= 22: return '20-22'
    elif age <= 25: return '22-25'
    elif age <= 29: return '25-29'
    elif age <= 34: return '30-34'
    elif age <= 39: return '35-39'
    elif age <= 44: return '40-44'
    elif age <= 49: return '45-49'
    else: return '50+'

df_detailed = df.copy()
df_detailed['Detailed_AgeGroup'] = df_detailed['Age'].apply(create_detailed_age_groups)

detailed_analysis = df_detailed.groupby(['Detailed_AgeGroup', 'Gender']).agg({
    'Profit': [
        ('Клиентов', 'count'),
        ('Средняя_прибыль', 'mean'),
        ('Медианная_прибыль', 'median'),
        ('Общая_прибыль', 'sum')
    ]
}).round(0)

print(detailed_analysis)

                           Profit                                    \
                         Клиентов Средняя_прибыль Медианная_прибыль   
Detailed_AgeGroup Gender                                              
18-20             female     2042         25100.0           24950.0   
                  male       3150          2042.0            2050.0   
20-22             female      505         29705.0           29800.0   
                  male        398         79161.0           79200.0   
22-25             female      290         20801.0           21725.0   
                  male        206         53527.0           70500.0   
25-29             female      402          7941.0            7575.0   
                  male        361          7789.0            8150.0   
30-34             female      433          7629.0            7600.0   
                  male        393         24959.0            8250.0   
35-39             female      252          8071.0            8350.0   
      